In [1]:
from Bio import Phylo
import numpy as np
import pandas as pd

### Day's algorithm
Due to the sorting algorithm it might run in O(nlogn)

In [5]:
def DAY(t, t1):
    
    # Get random root
    root = t.get_terminals()[0].name
    
    # Root trees with the same root
    t.root_with_outgroup({"name": root})
    t1.root_with_outgroup({"name": root})
    
    # Depth first traversal of names in rooted trees
    terminals  = [x.name for x in t.get_terminals()]
    terminals1 = [x.name for x in t1.get_terminals()]
    
    # Assigning numbers corresponding to names in one of the trees
    DF         = {}
    for i in range(len(terminals)):
        DF[terminals[i]] = i
    
    # Recursively run Day's algorithm in depth first manner
    NODES  = day_nodes(DF, t.clade, [])
    NODES1 = day_nodes(DF, t1.clade, [])
    
    # Return two lists of lists
    return NODES, NODES1
    
def day_nodes(DF, c, NODES):
    if c.is_terminal() == False:
        
        # Get terminals to the given clade
        l = [DF[x.name] for x in c.get_terminals()]
        l.sort()
        NODES.append(l)
        
        # Follow each of the branches in the bifurcating node
        day_nodes(DF, c.clades[0], NODES)
        day_nodes(DF, c.clades[1], NODES)
    
    if len(NODES) == len(c.get_nonterminals()):
        return NODES
    
def RF_dist(tree1, tree2):
    
    # Calculates the number of different lists between the two lists of lists
    
    l = DAY(tree1, tree2)
    score = 0
    for i in l[0]:
        if i in l[1]:
            score += 1
    return len(l[0]) + len(l[1]) - 2*score

### 6 trees, same data, different algorithms

Aligning: Clustal Omega, Muscle, and Kalign

Tree algorithm: Quick tree and Rapid NJ

In [6]:
trees = ["clustalo_tree_rapid_NJ.NEW", "kalign_tree_rapid_NJ.NEW", 
         "muscle_tree_rapid_NJ.NEW", "clustalo_quicktree.NEW", 
         "kalign_quicktree.NEW", "muscle_quicktree.NEW"]

path  = "/Users/johan/Dokumenter/Week8/Week8/trees/"

n_trees = len(trees)
RF_matrix = np.zeros((n_trees, n_trees))

for i in range(n_trees):
    for j in range(n_trees):
        if i > j and i != j:
            tree_i = Phylo.read(path+trees[i], "newick")
            tree_j = Phylo.read(path+trees[j], "newick")
            RF_matrix[i, j] = RF_dist(tree_i, tree_j)
            
RF_matrix = RF_matrix.T + RF_matrix

### Results

In [7]:
result = pd.DataFrame(RF_matrix)
result.columns = ["ClustalO RapidNJ", "Kalign RapidNJ", 
         "Muscle RapidNJ", "Clustalo Quicktree", 
         "Kalign Quicktree", "Muscle Quicktree"]
result.index = ["ClustalO RapidNJ", "Kalign RapidNJ", 
         "Muscle RapidNJ", "Clustalo Quicktree", 
         "Kalign Quicktree", "Muscle Quicktree"]
result

,ClustalO RapidNJ,Kalign RapidNJ,Muscle RapidNJ,Clustalo Quicktree,Kalign Quicktree,Muscle Quicktree
ClustalO RapidNJ,0.0,200.0,242.0,234.0,260.0,288.0
Kalign RapidNJ,200.0,0.0,218.0,228.0,202.0,274.0
Muscle RapidNJ,242.0,218.0,0.0,256.0,266.0,206.0
Clustalo Quicktree,234.0,228.0,256.0,0.0,156.0,206.0
Kalign Quicktree,260.0,202.0,266.0,156.0,0.0,194.0
Muscle Quicktree,288.0,274.0,206.0,206.0,194.0,0.0
